## 4-2　条件に応じた結合テーブルの切り替え

In [1]:
import gc
import numpy as np
import pandas as pd

In [2]:
hotel_tb = pd.read_csv('hotel.csv')
print(hotel_tb.shape)
hotel_tb.head()

(300, 7)


,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,h_1,26100,D,D-2,43.064569,141.511397,True
1,h_2,26400,A,A-1,35.715320,139.939446,True
2,h_3,41300,E,E-4,35.281572,136.988565,False
3,h_4,5200,C,C-3,38.431293,140.795615,False
4,h_5,13500,G,G-3,33.597291,130.533872,True


推薦される側のテーブルを作成する。

In [3]:
small_area_mst = hotel_tb.groupby(['big_area_name', 'small_area_name']).size().reset_index()
small_area_mst.columns = ['big_area_name', 'small_area_name', 'hotel_cnt']
small_area_mst.head()

,big_area_name,small_area_name,hotel_cnt
0,A,A-1,35
1,A,A-3,30
2,B,B-1,15
3,B,B-2,18
4,B,B-3,19


In [4]:
small_area_mst['join_area_id'] = np.where(small_area_mst['hotel_cnt']-1 <= 20,
                                          small_area_mst['small_area_name'], small_area_mst['big_area_name'])
small_area_mst.drop(['hotel_cnt', 'big_area_name'], axis=1, inplace=True)
small_area_mst.head()

,small_area_name,join_area_id
0,A-1,A
1,A-3,A
2,B-1,B-1
3,B-2,B-2
4,B-3,B-3


In [5]:
base_hotel_mst = pd.merge(hotel_tb, small_area_mst, on='small_area_name').loc[:, ['hotel_id', 'join_area_id']]
print(base_hotel_mst.shape)
base_hotel_mst.head()

(300, 2)


,hotel_id,join_area_id
0,h_1,D-2
1,h_79,D-2
2,h_125,D-2
3,h_127,D-2
4,h_129,D-2


In [6]:
del small_area_mst
gc.collect()

15

推薦先のテーブルを作成する。

In [7]:
recommend_hotel_mst = pd.concat([
    hotel_tb[['small_area_name', 'hotel_id']].rename(columns={'small_area_name': 'join_area_id'}, inplace=False),
    hotel_tb[['big_area_name', 'hotel_id']].rename(columns={'big_area_name': 'join_area_id'}, inplace=False)
])
recommend_hotel_mst.rename(columns={'hotel_id': 'rec_hotel_id'}, inplace=True)
print(recommend_hotel_mst.shape)
recommend_hotel_mst.head()

(600, 2)


,join_area_id,rec_hotel_id
0,D-2,h_1
1,A-1,h_2
2,E-4,h_3
3,C-3,h_4
4,G-3,h_5


両テーブルを結合する。

In [8]:
df = pd.merge(base_hotel_mst, recommend_hotel_mst, on='join_area_id').loc[:, ['hotel_id', 'rec_hotel_id']].query('hotel_id != rec_hotel_id')
print(df.shape)
df.head()

(10227, 2)


,hotel_id,rec_hotel_id
1,h_1,h_79
2,h_1,h_125
3,h_1,h_127
4,h_1,h_129
5,h_1,h_196
